# Creating HAWC bioassay data via the HAWC client

This notebooks demonstrates using the HAWC client to programmatically create animal bioassay endpoints in HAWC.

Make sure the `hawc_client` is installed. If you need to install it:

```bash
pip install -U hawc_client
```

Then, we'll create all the components for a assay, from experiment to individual endpoint. This assumes you have write permission and a HAWC study has been created which is ready for extraction.

In [ ]:
from getpass import getpass

from hawc_client import HawcClient

First, setup a HAWC client instance and authenticate with your username and password:

In [ ]:
client = HawcClient('http://127.0.0.1:8000')
client.authenticate(email='webmaster@hawcproject.org', password=getpass())

## Creating experiments

Next, we'll create an experiment. The are a number of options available for metadata to add, and at this point the only way to determine what options are available are by reading the HAWC model source code. Hopefully we'll have useful utilities in the future which can provide more details on the metata. 

Feel free to [contact us](https://hawcproject.org/contact/) if you get stuck.

We'll create an experiment:


In [ ]:
data = dict(
    study_id=100517534,     
    name="30 day oral",     
    type="St",
    has_multiple_generations=False,
    chemical="2,3,7,8-Tetrachlorodibenzo-P-dioxin",
    cas="1746-01-6",
    chemical_source="ABC Inc.",
    purity_available=True,
    purity_qualifier="≥",
    purity=99.9,
    vehicle="DMSO",
    guideline_compliance="not reported",
    description="Deatils here."
)
experiment = client.animal.create_experiment(data)
experiment['id']

# Creating animal groups and dosing regimes

Now that we've created an experiment, we can associate an animal-group and dosing regime with the experiment.

In many cases, the animal-groups being observed will have an associated dosing regime applied.

In [ ]:
data = dict(
    experiment_id=experiment['id'],     
    name="Female C57BL/6 Mice",
    species=2,
    strain=6,
    sex="F",
    animal_source="Charles River",
    lifestage_exposed="Adult",
    lifestage_assessed="Adult",
    generation="",
    comments="Detailed comments here",
    diet="...",
    dosing_regime=dict(
        route_of_exposure= "OR",
        duration_exposure=30,
        duration_exposure_text="30 days",
        duration_observation=180,
        num_dose_groups=3,
        positive_control=True,
        negative_control="VT",
        description="...",
        doses = [
            {"dose_group_id": 0, "dose": 0, "dose_units_id": 1},
            {"dose_group_id": 1, "dose": 50, "dose_units_id": 1},
            {"dose_group_id": 2, "dose": 100, "dose_units_id": 1},
            {"dose_group_id": 0, "dose": 0, "dose_units_id": 2},
            {"dose_group_id": 1, "dose": 3.7, "dose_units_id": 2},
            {"dose_group_id": 2, "dose": 11.4, "dose_units_id": 2},
        ],
    )
)
animal_group1 = client.animal.create_animal_group(data)
animal_group_id = animal_group['id']
dosing_regime_id = animal_group['dosing_regime']['id']

However, for developmental/reproductive studies, you can also specify a dosing regime which was applied to another group: 

In [ ]:
data = dict(
    experiment_id=experiment['id'],     
    name="F1 Male/Female C57BL/6 Mice",
    species=2,
    strain=6,
    sex="C",
    animal_source="Charles River",
    lifestage_exposed="Adult",
    lifestage_assessed="Adult",
    dosing_regime_id=dosing_regime_id,
    generation="F1",
    comments="Detailed comments here",
    diet="...",
)
animal_group2 = client.animal.create_animal_group(data)

> TODO - add details on how to creating `animal_groups` with `parents` and `siblings` relations

## Creating endpoints